# 긍정 리뷰를 이용하여 긍정 키워드 추출
- 앞선 과정을 통해 긍정 리뷰와 부정 리뷰로 나누었고 konlpy를 이용하여 단어문서행렬을 만든 후 LDA를 이용하여 주제분석을 하여 키워드를 추출할 것이다.
- Stanza와 Konlpy 의 결과를 비교하였더니 Stanza의 경우 리뷰에 특수문자를 같이 포함시키고. 띄어쓰기가 되어 있지 않을 경우 전체를 명사로 뽑는 단점이 있다. Konlpy의 경우 리뷰의 품사 추출을 잘 하지만, 너무 세세하게 추출한다. 따라서 konlpy 를 이용하여 단어문서행렬을 만든 후 LDA를 통해 주제분석을 진행할 것이다.
- 우리의 목표는 호텔별 키워드 추출이다. 즉 호텔(=문서)의 해석을 위해 LDA를 이용하였다.

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
data=pd.read_excel('X_train.xlsx',encoding='cp949')

In [4]:
test=pd.read_excel('X_test.xlsx',encoding='cp949')

### 데이터셋 구축

In [5]:
data=data.query("평점==1")

In [6]:
test=test.query("평점==1")
test=test.reset_index(drop=True)

In [7]:
data=data.reset_index(drop=True)

In [8]:
data['호텔이름'].nunique()

296

In [9]:
data_dict=dict()
for i in range(len(data)):
    if data['호텔이름'][i] not in data_dict.keys():
        data_dict[data['호텔이름'][i]]=data['리뷰'][i]
    else:
        data_dict[data['호텔이름'][i]]=data_dict[data['호텔이름'][i]]+' '+ data['리뷰'][i]

In [10]:
test_dict=dict()
for i in range(len(test)):
    if test['호텔이름'][i] not in test_dict.keys():
        test_dict[test['호텔이름'][i]]=test['리뷰'][i]
    else:
        test_dict[test['호텔이름'][i]]=test_dict[test['호텔이름'][i]]+' '+ test['리뷰'][i]

In [11]:
a=pd.DataFrame.from_dict([data_dict])
final_data=pd.melt(a,var_name='호텔이름', value_name='리뷰')

In [12]:
b=pd.DataFrame.from_dict([test_dict])
final_test=pd.melt(b,var_name='호텔이름', value_name='리뷰')

In [13]:
len(final_data['호텔이름'].unique())

296

In [14]:
final_test

,호텔이름,리뷰
0,플레이스 캠프 제주,생각보다 홍보하는 것처럼 신나지는 않은 것 같습니다저희가 밤 10시쯤 도착한 것도있...
1,메리어트 이그제큐티브 아파트먼트 서울,직원분들이 매우 친절하여 들어갈때부터 기분 좋았으며 시설도 주방 포함된 레지던스 타...
2,오션팰리스 호텔,서귀포 시내에 있어서 이동이 좋았고 로비에 있는 편의점이 24시간이라서 이용하기 편...
3,신라스테이 서대문,가격대비 정말 훌륭한 호텔이었어요 전반적으로 깔끔하고 친절하고 좋았음 객실 화장실 ...
4,신라스테이 제주,"깨끗하고 청결합니다 주차도 편하게 되어있어요 위치가 좋은 전망을 기대하긴 어렵지만,..."
...,...,...
172,노보텔 앰배서더 서울 동대문 호텔 & 레지던스,"태풍 링링의 영향으로 호텔 체크인전 일정이 모두 취소됬는데, 직원분들이 친절하게 안..."
173,골드원 호텔 & 스위트,전망이 너무 좋고 매트리스도 편안하네요 서귀포 혁신도시에 일이 있어 와이프랑 가끔 ...
174,라마다 군산호텔,새 건물이었고 객실 관리도 잘 되어 있고 조식도 좋았어요 새 건물이었고 객실 관리도...
175,하워드 존슨 호텔 인천 에어포트,인천공항 근처의 스톱오버 전용 호텔인듯 많은 항공사 직원의 숙소임 2개의 호텔이 연...


# Konlpy 로 TDM 만들기

In [48]:
from konlpy.tag import Komoran

In [49]:
tagger = Komoran()

In [60]:
def extract_nouns(text):
    nouns = tagger.nouns(text)
    values = [noun for noun in nouns if (len(noun)> 1 and noun!='호텔') or noun == '돈' ]
    return values

In [61]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=500, tokenizer= extract_nouns)
tdm = cv.fit_transform(final_data['리뷰'])

In [62]:
tdm_test = cv.transform(final_test['리뷰'])

In [63]:
from gensim.matutils import Sparse2Corpus

In [64]:
corpus = Sparse2Corpus(tdm.T) 
id2word = dict(enumerate(cv.get_feature_names()))

In [65]:
corpus_test = Sparse2Corpus(tdm_test.T)

## LDA 

- SVD

In [66]:
from gensim.models.ldamodel import LdaModel

In [67]:
for num_topics in [3, 4, 5 , 6]:
    globals()['models_{}'.format(num_topics)] = LdaModel(
        corpus=corpus,
        num_topics=num_topics,
        passes=3,
        iterations=200,
        id2word=id2word,
        random_state=123)

2020-05-13 05:35:14 WARNING: too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2020-05-13 05:35:19 WARNING: too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2020-05-13 05:35:25 WARNING: too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2020-05-13 05:35:31 WARNING: too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy


In [136]:
for i in [3, 4 , 5 , 6]:
    log_pp = globals()['models_%s'%().format(i)].log_perplexity(corpus_test)
    coh = CoherenceModel(model=globals()['models_{}'.format(i)], corpus=corpus_test, coherence='u_mass')
    print(i, globals()['models_{}'.format(
        i)],"혼란도: ",2 ** (-log_pp),"응집도: ",coh.get_coherence())

3 LdaModel(num_terms=500, num_topics=3, decay=0.5, chunksize=2000) 혼란도:  57.07194844862846 응집도:  -1.5649170137948964
4 LdaModel(num_terms=500, num_topics=4, decay=0.5, chunksize=2000) 혼란도:  61.81236189804071 응집도:  -1.450837971076801
5 LdaModel(num_terms=500, num_topics=5, decay=0.5, chunksize=2000) 혼란도:  66.68232230815467 응집도:  -1.9924325493503325
6 LdaModel(num_terms=500, num_topics=6, decay=0.5, chunksize=2000) 혼란도:  69.57850858812479 응집도:  -2.1145905789456534


#### 3 topics Model Selection

In [70]:
for words in models_3.show_topics(formatted=False,num_words=15, num_topics = 3):
    print(words[0])
    print("******************************")
    for word_prob in words[1]:
        print(word_prob)
    print("")
    print("******************************")

0
******************************
('만족', 0.029515484)
('친절', 0.026728919)
('직원', 0.026120877)
('위치', 0.025307467)
('조식', 0.023444502)
('이용', 0.020618234)
('시설', 0.02002133)
('서비스', 0.017228976)
('가격', 0.014832828)
('청결', 0.014066408)
('주변', 0.013461048)
('객실', 0.013273926)
('전반', 0.012455535)
('숙박', 0.011655783)
('침대', 0.011285818)

******************************
1
******************************
('친절', 0.035501566)
('만족', 0.031197352)
('직원', 0.025026128)
('이용', 0.024646811)
('위치', 0.019380936)
('시설', 0.017302286)
('객실', 0.017273618)
('조식', 0.015989324)
('주변', 0.015096155)
('가격', 0.014758016)
('서비스', 0.013397333)
('수영장', 0.013336449)
('숙박', 0.012581023)
('편안', 0.011819507)
('생각', 0.011243197)

******************************
2
******************************
('위치', 0.032459177)
('서울역', 0.022413796)
('가격', 0.020543767)
('화장실', 0.019217247)
('객실', 0.018114377)
('시설', 0.01762525)
('만족', 0.014166781)
('친절', 0.013512537)
('침대', 0.013306671)
('이용', 0.012798624)
('대비', 0.012537808)
('생각', 0.01253

### 테스트 데이터에 적용

In [71]:
topic=[]
for i in range(len(corpus)):
    doc_topic_dist = models_3.get_document_topics(corpus[i], minimum_probability=0)
    sorted_doc_topic = sorted(doc_topic_dist, key=lambda x:x[1], reverse=True)
    topic.append(sorted_doc_topic[0][0])

In [72]:
final_data['긍정']=topic

In [73]:
final_data['긍정'].value_counts()

0    156
1    115
2     25
Name: 긍정, dtype: int64

In [74]:
final_data.to_excel('final_data.xlsx',encoding='cp949',index=False)